# Cryptocurrency Weekly Strategy

By Nicolas Harvie

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

import datetime

***
## Universe Scraper 
Grab top 20 market cap cryptocurrencies (scraped from CoinMarketCap)

In [2]:
###=== UNIVERSE SCRAPER ===###

url = "https://coinmarketcap.com/"
response = requests.get(url)

soup = BeautifulSoup(response.content, "html.parser")
table = soup.find("table")

rows = table.find_all("tr")

ticker_list = []

for i, row in enumerate(rows[1:101]): # exclude header row
    if i < 10:
        ticker = row.find("a", {"class": "cmc-link"}).find('p', {'class': 'coin-item-symbol'}).text
    else:
        ticker = row.find("a", {"class": "cmc-link"}).find("span", {"class": "crypto-symbol"}).text

    ticker_list.append(ticker)

In [3]:
###=== CLEANING AND SELECTION OF THE UNIVERSE ===###

no_trades = ['USD', 'DAI', 'WBTC', 'WETH']

for i in range(len(no_trades)):    
    ticker_list = [x for x in ticker_list if no_trades[i] not in x]

top_twenty_mktcap = ticker_list[:20]

***
## Price Scraper

In [4]:
# Returns for past 14 days
start_date = (datetime.date.today() - datetime.timedelta(14)).strftime('%d-%m-%Y')
end_date = (datetime.date.today()).strftime('%d-%m-%Y')

### With CMC Scraper

In [5]:
from cryptocmd import CmcScraper

# Inital DF with Bitcoin 
scraper = CmcScraper("BTC", start_date, end_date)
df = pd.DataFrame(scraper.get_dataframe(date_as_index=True)['Close']).rename(columns={'Close':'BTC'})

for crypto in top_twenty_mktcap[1:]:
    scraper = CmcScraper(str(crypto), start_date, end_date)
    df = pd.merge(df, pd.DataFrame(scraper.get_dataframe(date_as_index=True)['Close']).rename(columns={'Close':str(crypto)}), left_index=True, right_index=True)
price_df = df[::-1]


***
## Strategy itself

In [6]:
def output_strategy(n_ls, data, output_sort=False):

    # Cumulative Returns for the Past Week
    log_ret_df = np.log(1 + data.pct_change())
    cum_returns = log_ret_df.apply(lambda x: x.sum())

    if output_sort == True:
        print(cum_returns.sort_values(ascending=False))
    

    top_stocks = cum_returns.sort_values(ascending=False).index[:n_ls].values
    bottom_stocks = cum_returns.sort_values(ascending=True).index[:n_ls].values

    return top_stocks, bottom_stocks


In [7]:
to_long, to_short = output_strategy(3, price_df, output_sort=True)

TON      0.215408
SHIB     0.197211
SOL      0.031926
BTC      0.000631
BCH     -0.001749
TRX     -0.002335
LEO     -0.005432
ETH     -0.006632
BNB     -0.015991
XMR     -0.029121
UNI     -0.031761
LINK    -0.043069
DOGE    -0.046198
DOT     -0.049667
AVAX    -0.060097
ADA     -0.060484
MATIC   -0.064112
LTC     -0.098607
XLM     -0.100499
XRP     -0.116902
dtype: float64


In [8]:
available_gmx = ['ETH', 'BTC', 'LINK', 'ARB', 'UNI', 'SOL', 'DOGE', 'XRP', 'LTC']

In [9]:
gmx_to_short = [value for value in to_short if value in available_gmx]
gmx_to_long = [value for value in to_long if value in available_gmx]

print(f'To long {gmx_to_long}')
print(f'To short {gmx_to_short}')

To long ['SOL']
To short ['XRP', 'LTC']
